# CounterFactual Optimization for Bidding

In [1]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


In [213]:
from scipy.stats import lognorm

In [ ]:
from sklearn.linear_model import LinearRegression

## Dataset Exploration

Read the dataset. `action` is the action and `y` is the reward. The action is a randomization around the deterministic bid and the reward is the value of a sale (if there is any for this user) minus the cost of the displays.

1. Estimate the mean of the action under the current policy
1. Do a linear regression on `y ~ action`. What do you observe ?
1. Estimate the expectation of the reward by bucket of the action. What do you observe ? Look also at the variance in each bucket.
1. Do you think it is possible to propose a better policy ? E.g. by bidding above/under the current average action ?

In [265]:
df = pd.read_csv('criteo-continuous-bandit-dataset.csv.1M.gz', compression='gzip')

## Counterfactual Reasoning on the bid level

The randomization (action) follows a lognormal distribution, which parameters we'll learn now.

In [214]:
mle_params = lognorm.fit(df.action)
pa = lognorm(s=mle_params[0], loc=mle_params[1], scale=mle_params[2])

We will consider a family of alternative policies instantiated as lognormal distributions with a different mean $\alpha$. Our parameter $\alpha$ will be restricted to the neighboorhood of the current policy e.g. [.8, 1.2], meaning that on average we would multiply our bids by this $\alpha$ coefficient. 

NB: $\alpha$ is not a direct parameter of the lognormal distribution, the code below computes the relevant $\mu$ value so that $E[\pi_{\mu}] = \alpha$

1. Execute the code below to instantiate the family of policies $\{\pi_\alpha\}_{\alpha \in [.8;1.2]}$
1. Plot $\mu$ wrt $\alpha$ 
1. Plot $w$, the corresponding importance weights. What property should they respect? Is that the case for all $\alpha$? In your opinion, what should we do ? 

In [227]:
alphas = np.linspace(.8, 1.2, 10)
mus = []
avgs = []
ws = []
for alpha in alphas:
    mu_i = np.log(alpha) - mle_params[0]/2
    mus += [mu_i]
    pi = lognorm(s=mle_params[0], loc=mle_params[1], scale=np.exp(mu_i))
    ws += [np.mean(pi.pdf(df.action)/pa.pdf(df.action))]
    avg = pi.rvs(size=10**5).mean()
    avgs += [avg]

### Zero-order optimization of the bid level

The goal now is to find the best possible policy (= best $\alpha$ wrt $E_{\pi_\alpha}[Y]$) by grid-search over the possible values. For this we will use the following code that implements importance sampling, including estimation of the variance of the reward.

1. choose an appropriate grid for $\alpha$
1. compute the corresponding parameters of the lognormal as in the previous section
1. estimate $E_{\pi_\alpha}[Y]$ using the code; do you find a better policy ?
1. pick your best $\alpha$ and use `nb_bootstraps=30` to generate a distribution of the reward; transform it into a confidence interval for $E_{\pi_\alpha}[Y]$. Is the reward of the original policy inside ?
1. redo the same but use the clipping parameter, e.g. ̀̀`clip=1000`; what do you observe ? are you confident on the result ?

In [219]:
def clip_weights(w, clip='bottou', verbose=False):
    w_ = w.copy()
    if clip is not None:
        if clip == 'bottou':
            clip = float(np.sort(w)[-5])
        elif type(clip) in (int, float):
            w = np.clip(w, None, clip)    
        else:
            raise ValueError('unsupported clipping value:', clip)
    if verbose:
        print("clip rmse:", np.sqrt(np.mean((w - w_)**2)))
    return w

def evaluate_policy(pi, pi0, y, clip=None, bootstraps:int=None, self_normalized=True, verbose=False):
    """
    Estimate E_{\pi}[Y] using importance sampling.
    
    - pi = proba to take action under new policy (vector of size n_samples)
    - pi0 = proba to take action under original policy (vector of size n_samples)
    - y = historical rewards (vector of size n_samples)
    
    if nb_bootstraps is not None, a bootstrap distribution of E_{\pi}[Y] is returned (array of size nb_bootstraps)
    otherwise the mean is returned (a scalar)
    """
    
    w = pi / pi0
    
    normalizer = lambda x: x / np.mean(w) if self_normalized else x

    if verbose:
        print("E[pi0]:", np.mean(pi0))
        print("E[pi']:", np.mean(pi))
        print("E[W]:", np.mean(w))
        print("E[W~]:", np.mean(normalizer(w)))
    
    w = normalizer(clip_weights(w, clip=clip, verbose=verbose))
    
    if bootstraps is not None:
        return [
                np.dot(
                    np.multiply(
                        w,
                        np.random.poisson(lam=1, size=len(y))
                    ), 
                    y
                ) / len(y)
            for _ in range(bootstraps)
        ]
    else:
        return np.dot(w, y) / len(df)

Concluding remarks:
- this procedure could be turned into a real optimization (e.g. by computing the gradient of $\alpha$)
- it could be made contextual by optimizing a different policy for each context 